<a href="https://colab.research.google.com/github/leandrobarbieri/pydata-book/blob/2nd-edition/Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas

In [ ]:
# Importando os pacotes/modulos do pandas
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt

# Configurações inicias das bibliotecas
np.random.seed(1234)
plt.rc('figure', figsize=(10, 6))

# salva a quantidade original de linhas exibidas
PREVIOUS_MAX_ROWS = pd.options.display.max_rows

# set a nova quantidade de itens máximo exibidos ao dar um display em um dataframe, usar ... para representar os itens intermediarios
pd.options.display.max_rows = 20

# set o formato de apresentação
np.set_printoptions(precision=4, suppress=True)


## Series

### index e values

In [ ]:
# Series do pandas é como um array numpy com um indice que pode conter labels
serie1 = pd.Series([4, 5, 6, -2, 2])

# atribui um indice automaticamente
print(f"{serie1}\n")

# index é um obj do tipo range com inicio, fim, step
print(f"Tipo: {type(serie1.index)}\n")

index = [x for x in serie1.index]
print(f"Somente o index: {index}\n")

0    4
1    5
2    6
3   -2
4    2
dtype: int64

Tipo: <class 'pandas.core.indexes.range.RangeIndex'>

Somente o index: [0, 1, 2, 3, 4]



In [ ]:
# definindo lables para o indice
serie2 = pd.Series([1, 2, 4, 5], index=["a", "b", "c", "d"])
print(f"Labels como índice:\n{serie2}\n")
print(f"Indice: {serie2.index} \nValores: {serie2.values}")


Labels como índice:
a    1
b    2
c    4
d    5
dtype: int64

Indice: Index(['a', 'b', 'c', 'd'], dtype='object') 
Valores: [1 2 4 5]


### slice e atribuição

In [ ]:
# um item específico
serie2["a"]

1

In [ ]:
# um range de itens. Limite superior incluido (diferente de indices numpy)
serie2["a":"c"]

a    1
b    2
c    4
dtype: int64

In [ ]:
# itens específicos em ordem específica
serie2[["b", "d", "c"]]

b    2
d    5
c    4
dtype: int64

In [ ]:
# atribuir valores
serie2[["d", "b"]] = 999
print(serie2)

a      1
b    999
c      4
d    999
dtype: int64


In [ ]:
# operações logicas nas series retornam resultados fitrados
serie_nova = serie2[serie2 < 10]
serie_nova

a    1
c    4
dtype: int64

In [ ]:
# operação vetorizada
serie_nova * 2

a    2
c    8
dtype: int64

In [ ]:
# verificar se um item está na serie (index)

# Retorna True/false
print(f"a in serie2: {'a' in serie2}")

# Retona o elemento
print(f"serie2['a']: {serie2['a']}")

# Retona chave/valor
print(f"serie2[serie2.index == 'a']: {serie2[serie2.index == 'a']}")

a in serie2: True
serie2['a']: 1
serie2[serie2.index == 'a']: a    1
dtype: int64


### pareamento a partir do index

In [ ]:
# Criar uma serie a partir de um dict
estados_dict = {"Sao Paulo": 45000, "Rio de Janeiro": 50141, "Espirito Santo": 30914}

# os estados são transformados em index e os valores e values
serie3 = pd.Series(estados_dict)

print(f"serie3:\n{serie3}\n")
print(f"index:\n{serie3.index}\n")
print(f"values:\n{serie3.values}\n")


serie3:
Sao Paulo         45000
Rio de Janeiro    50141
Espirito Santo    30914
dtype: int64

index:
Index(['Sao Paulo', 'Rio de Janeiro', 'Espirito Santo'], dtype='object')

values:
[45000 50141 30914]



In [ ]:
# definindo a propria lista de index. 

# A lista tem apenas 2 estados, ES ficou de fora 
estados_index = ["Sao Paulo", "Rio de Janeiro"]

# apesar dos dados terem o ES, ele não aparece porque foi definido apenas SP e RJ na lista de indices
serie4 = pd.Series(estados_dict, index=estados_index)
serie4

Sao Paulo         45000
Rio de Janeiro    50141
dtype: int64

In [ ]:
# se um index não existir na serie de dados, será preenchido com NaN
# É como se um left join entre os indices informados no parametro index de series e os indices preexistentes na serie de dados
serie5 = pd.Series(data=estados_dict, index=["Sao Paulo", "Rio de Janeiro", "Novo Estado"])
serie5

Sao Paulo         45000.0
Rio de Janeiro    50141.0
Novo Estado           NaN
dtype: float64

In [ ]:
# descobrir se existe elementos null na serie
# serie5.isnull()
pd.isnull(serie5)

Sao Paulo         False
Rio de Janeiro    False
Novo Estado        True
dtype: bool

In [ ]:
# Retorna somente itens null
serie5[pd.isnull(serie5)]

Novo Estado   NaN
dtype: float64

In [ ]:
# somar series faz o pareamento pelo indice
# na serie3 existe ES mas na serie5 não
# Novo Estado existe na serie5 mas não na serie3
# qualquer um que tenha NaN vai gerar um NaN na soma final, mesmo que exista algum valore
serie3 + serie5

Espirito Santo         NaN
Novo Estado            NaN
Rio de Janeiro    100282.0
Sao Paulo          90000.0
dtype: float64

In [ ]:
# para somar sem perder o valor caso tenha algum valor NaN na soma
# o ES agora retorna a população mesmo com uma soma com NaN (porque serie5 não tem ES)
serie3.add(serie5, fill_value=0)

Espirito Santo     30914.0
Novo Estado            NaN
Rio de Janeiro    100282.0
Sao Paulo          90000.0
dtype: float64

In [ ]:
# metadado da serie, identifica o nome para a serie
serie5.name = "Populacao"
# nome do index
serie5.index.name = "Estados"

# renomeando os indices
# serie5.index = ["a", "b", "c"]

# alterando o nome da coluna de index
serie5.index.rename('UF', inplace=True)

serie5

UF
Sao Paulo         45000.0
Rio de Janeiro    50141.0
Novo Estado           NaN
Name: Populacao, dtype: float64

## DataFrames
São tabelas com linhas e colunas que podem ser indexadas com os parametros index ou columns.
Vários tipos de objetos podem ser transformados em DataFrames (lists, dicts..)
Possuem uma sério de funções para manipulação dos dados e filtragem

### criando um dataframe a partir de um dict

In [ ]:
# são como series multidimensionais que compartilham o mesmo index
# possuem index/labels nas linhas (index=) e colunas (columns=)

# criando um dataframe a partir de um dict
dados1 = {"estado": ["SP", "RJ", "ES"] * 3, 
          "ano": [2000, 2000, 2000, 2010, 2010, 2010, 2020, 2020, 2020],
          "populacao": [50_000, 30_000, 20_000, 55_000, 33_000, 22_000, 60_000, 40_000, 30_000]}

# transforma um dicionario em um dataframe
df1 = pd.DataFrame(dados1)
df1

,estado,ano,populacao
0,SP,2000,50000
1,RJ,2000,30000
2,ES,2000,20000
3,SP,2010,55000
4,RJ,2010,33000
5,ES,2010,22000
6,SP,2020,60000
7,RJ,2020,40000
8,ES,2020,30000


### listar dataframes

In [ ]:
# alguns metodos úteis para listar dataframes

# top5
df1.head(5)
#df1[:5]

# ultimos 2
# df1.tail(2)
# df1[-2:]

# lista tudo
#display(df1)

,estado,ano,populacao
0,SP,2000,50000
1,RJ,2000,30000
2,ES,2000,20000
3,SP,2010,55000
4,RJ,2010,33000


In [ ]:
# lista algumas colunas
df1[["estado", "populacao"]]

# estatisticas da populaçao
df1["populacao"].describe()

count        9.000000
mean     37777.777778
std      14376.871859
min      20000.000000
25%      30000.000000
50%      33000.000000
75%      50000.000000
max      60000.000000
Name: populacao, dtype: float64

### alterando a sequencia das colunas

In [ ]:
print(f"Sequência atual das colunas: {list(df1.columns)}\n")

# alterando a sequencia
df_sequencia_colunas_alteradas = pd.DataFrame(df1, columns=["ano", "estado", "populacao"])
df_sequencia_colunas_alteradas

Sequência atual das colunas: ['estado', 'ano', 'populacao']



,ano,estado,populacao
0,2000,SP,50000
1,2000,RJ,30000
2,2000,ES,20000
3,2010,SP,55000
4,2010,RJ,33000
5,2010,ES,22000
6,2020,SP,60000
7,2020,RJ,40000
8,2020,ES,30000


### criando colunas em df existente

In [ ]:
# criando um df novo com colunas e indices que ainda não exitem no dict (coluna dif e index 9)

df3 = pd.DataFrame(dados1, columns=["ano", "estado", "populacao", "dif"], 
                                          index=[0, 1, 2, 3, 4, 5, 6, 7, 8])


In [ ]:
df3

,ano,estado,populacao,dif
0,2000,SP,50000,NaN
1,2000,RJ,30000,NaN
2,2000,ES,20000,NaN
3,2010,SP,55000,NaN
4,2010,RJ,33000,NaN
5,2010,ES,22000,NaN
6,2020,SP,60000,NaN
7,2020,RJ,40000,NaN
8,2020,ES,30000,NaN


### filtrando linhas e colunas loc e iloc

In [ ]:
# localizando linhas e colunas usando loc e iloc

# filtrando uma coluna
dados_es = df3.loc[df3["estado"] == "ES", ["estado", "ano", "populacao"]]
dados_es
#print(dados_es[["ano", "estado", "populacao"]])

,estado,ano,populacao
2,ES,2000,20000
5,ES,2010,22000
8,ES,2020,30000


In [ ]:
# filtrando com iloc: usando indices para buscar os 3 primeiros registros e as duas primeiras colunas
dados_es = df3.iloc[:3, :2]
print(dados_es[["ano", "estado"]])

    ano estado
0  2000     SP
1  2000     RJ
2  2000     ES


#### filtrando uma coluna com mais de uma condição

In [ ]:
# filtrando uma coluna com mais de uma condição
dados_es_2010 = df3.loc[ (df3["ano"] == 2010) & (df3["estado"] == "ES") ]

print(dados_es_2010[["ano", "estado", "populacao"]])



    ano estado  populacao
5  2010     ES      22000


### preencher a nova coluna

In [ ]:
# preencher a nova coluna com dados incrementais de arange iniciando em zero e indo até o tamanho maximo da tabela
df3["dif"] = np.arange(len(df3))
df3

,ano,estado,populacao,dif
0,2000,SP,50000,0
1,2000,RJ,30000,1
2,2000,ES,20000,2
3,2010,SP,55000,3
4,2010,RJ,33000,4
5,2010,ES,22000,5
6,2020,SP,60000,6
7,2020,RJ,40000,7
8,2020,ES,30000,8


In [ ]:
# Cria uma serie independente somente com 3 valores
# Faz um left join da serie com o dataframe. Somente onde os indices da nova serie correspondem com o dataframe será inserido
valores_dif = pd.Series(np.random.randn(5))

# Preenche os 5 primeiros registros com os valores da serie
df3["dif"] = valores_dif

# completa os demais com o valor zero
df3.fillna(value=0)

,ano,estado,populacao,dif
0,2000,SP,50000,0.471435
1,2000,RJ,30000,-1.190976
2,2000,ES,20000,1.432707
3,2010,SP,55000,-0.312652
4,2010,RJ,33000,-0.720589
5,2010,ES,22000,0.000000
6,2020,SP,60000,0.000000
7,2020,RJ,40000,0.000000
8,2020,ES,30000,0.000000


In [ ]:
# Criando uma nova coluna a partir de um resultado lógico de outra
df3["Regiao"] = df3["estado"] == "ES"
df3

,ano,estado,populacao,dif,Regiao
0,2000,SP,50000,0.471435,False
1,2000,RJ,30000,-1.190976,False
2,2000,ES,20000,1.432707,True
3,2010,SP,55000,-0.312652,False
4,2010,RJ,33000,-0.720589,False
5,2010,ES,22000,NaN,True
6,2020,SP,60000,NaN,False
7,2020,RJ,40000,NaN,False
8,2020,ES,30000,NaN,True


In [ ]:
# dict de dicts de população dos estados
pop = {"ES": {2000: 22000, 2010: 24000},
       "RJ": {2000: 33000, 2010: 36000, 2020: 66000}}
# type(pop)

# pd.DataFrame(pop).T
df4 = pd.DataFrame(pop, index=[2000, 2010, 2020])
df4

,ES,RJ
2000,22000.0,33000
2010,24000.0,36000
2020,NaN,66000


In [ ]:
# remove a ultima linha de ES que tem NaN e considera apenas as dias primeiras de RJ
pdata = {"ES": df4["ES"][:-1], 
         "RJ": df4["RJ"][:2]}

# Define um nome para a coluna de indices e um nome para as colunas da tabela
df4= pd.DataFrame(pdata)
df4.index.name = "ANO"
df4.columns.name = "UF"
df4

UF,ES,RJ
ANO,,
2000,22000.0,33000
2010,24000.0,36000


## Objeto Index
São os objetos responsaveis por armazenar os rotulos dos eixos e outros metadados

In [ ]:
obj_index = pd.Series(np.arange(3), index=["a", "b", "c"])

# armazena o index da serie
index = obj_index.index

# pandas.core.indexes.base.Index
type(index)

# um index é sempre imutavel: TypeError: Index does not support mutable operations
# index["a"] = "x"


pandas.core.indexes.base.Index

In [ ]:
# criando um obj index
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [ ]:
# Usando um obj do tipo index para criar uma series
obj_Series2 = pd.Series([1.4, 3.5, 5.2], index=labels)
obj_Series2

0    1.4
1    3.5
2    5.2
dtype: float64

In [ ]:
print(f"df original:\n {df4}\n")

# verificando a existencia de uma coluna
print("ES" in df4.columns)

# verificando a existencia de um indice
print(2020 in df4.index)

df original:
 UF         ES     RJ
ANO                 
2000  22000.0  33000
2010  24000.0  36000

True
False


## Reindexação de linhas e colunas

Redefine o indice de um df já criado. 
Se os novos indices já existem serão mantidos senão serão introduzidos com valores faltantes NaN

In [ ]:
obj_Series3 = pd.Series(np.random.randn(4), index=["a", "b", "c", "d"])
obj_Series3

a    0.887163
b    0.859588
c   -0.636524
d    0.015696
dtype: float64

In [ ]:
# Os indices com os mesmos valores são mantidos, os novos recebem os valores NaN
obj_Series3 = obj_Series3.reindex(["a", "b", "c", "d", "x", "y", "z"])
obj_Series3

a    0.887163
b    0.859588
c   -0.636524
d    0.015696
x         NaN
y         NaN
z         NaN
dtype: float64

In [ ]:
obj_Series4 = pd.Series(["Azul", "Amarelo", "Verde"], index=[0, 2, 4])

# passando um range de 6 os valores auxentes no indice atual ficam com o valor NaN
obj_Series4.reindex(range(6))

0       Azul
1        NaN
2    Amarelo
3        NaN
4      Verde
5        NaN
dtype: object

In [ ]:
# usando o metodo ffill (forward fill) preenche os valores NaN com o valor anterior a ocorrencia do NaN
# é como um "preencher para frente"
obj_Series4.reindex(range(6), method="ffill")

0       Azul
1       Azul
2    Amarelo
3    Amarelo
4      Verde
5      Verde
dtype: object

In [ ]:
# reindexando linhas e colunas
df5 = pd.DataFrame(np.arange(9).reshape((3, 3)),
                   index=["a", "c", "d"],
                   columns=["ES","RJ", "SP"]
                   )
df5

,ES,RJ,SP
a,0,1,2
c,3,4,5
d,6,7,8


In [ ]:
# reindexar as linhas: inclui o indice "b"
# index=<default>
# columns=<lista de colunas>
df5 = df5.reindex(index=["a", "b", "c", "d"], columns=["ES","RJ", "SP", "MG"])
df5

,ES,RJ,SP,MG
a,0.0,1.0,2.0,NaN
b,NaN,NaN,NaN,NaN
c,3.0,4.0,5.0,NaN
d,6.0,7.0,8.0,NaN


In [ ]:
# se passar outra lista para indexação, irá ser feito um left join, os valores atuais serão mantidos e os novos serão inseridos com o valor NaN

novos_estados = ["ES","RJ", "SP", "MG", "RS", "PR", "BA"]

df5 = df5.reindex(columns=novos_estados)
df5

,ES,RJ,SP,MG,RS,PR,BA
a,0.0,1.0,2.0,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c,3.0,4.0,5.0,NaN,NaN,NaN,NaN
d,6.0,7.0,8.0,NaN,NaN,NaN,NaN


In [ ]:
# selecionando colunas a partir de uma lista
filtro_estados = ["ES","RJ", "SP"]
dados = df5.loc[:, filtro_estados]

### apagando linhas ou colunas

In [ ]:
# copiando dados do df anterior
dados_para_apagar = dados

# apagar linhas

# inplace True para de verdade, False (padrão), remove na memoria mas não apaga do obj original
dados_para_apagar.drop(["b", "d"], inplace=True)

dados_para_apagar

,ES,RJ,SP
a,0.0,1.0,2.0
c,3.0,4.0,5.0


In [ ]:
# apagar colunas axis=1
dados_sem_RJ_SP = dados_para_apagar.drop(["SP", "RJ"], axis=1)
dados_sem_RJ_SP

,ES
a,0.0
c,3.0


In [ ]:
dados_para_apagar.drop(["SP", "RJ"], axis="columns", inplace=True)
dados_para_apagar

,ES
a,0.0
c,3.0


## Seleção e Filtragem
a seleção e filtragem de df é semelhante a arrays numpy porém com o uso de rotulos nomeados nos dois eixos.

### Series

In [ ]:
# filtrando Series
obj_Series5 = pd.Series(np.arange(4.0), index=["a", "b", "c", "d"])
obj_Series5

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [ ]:
# rótulo da linha
obj_Series5["d"]

3.0

In [ ]:
# índice da linha: implicitamente o pandas cria um índice mesm o que exista um rótulo
obj_Series5[3]

3.0

In [ ]:
# intervalo do ínidice implicito 2 ate o 4
obj_Series5[2:4]

c    2.0
d    3.0
dtype: float64

In [ ]:
# diferentemente do acesso pelo índice implicito, o indice pelo rotulo sempre inclui o ultimo elemento
obj_Series5["b":"d"]

b    1.0
c    2.0
d    3.0
dtype: float64

In [ ]:
# elementos específicos
obj_Series5[["a", "c", "d"]]

a    0.0
c    2.0
d    3.0
dtype: float64

In [ ]:
# filtrando usando uma condição
obj_Series5[obj_Series5 > 2]

d    3.0
dtype: float64

In [ ]:
# atribuindo valores a uma faixa de indices
obj_Series5["b":"d"] = 4
obj_Series5

a    0.0
b    4.0
c    4.0
d    4.0
dtype: float64

### DataFrame

In [ ]:
df6 = pd.DataFrame(np.random.randn(16).reshape(4, 4), 
                   index=["ES", "RJ", "SP", "MG"],
                   columns=[2000, 2010, 2020, 2030])
print("Dados Iniciais:\n")
df6
                   

Dados Iniciais:



,2000,2010,2020,2030
ES,-2.242685,1.150036,0.991946,0.953324
RJ,-2.021255,-0.334077,0.002118,0.405453
SP,0.289092,1.321158,-1.546906,-0.202646
MG,-0.655969,0.193421,0.553439,1.318152


In [ ]:
# uma coluna específica
df6[2020]

ES    0.991946
RJ    0.002118
SP   -1.546906
MG    0.553439
Name: 2020, dtype: float64

In [ ]:
# subconjunto de colunas
df6[[2010, 2020]]

,2010,2020
ES,1.150036,0.991946
RJ,-0.334077,0.002118
SP,1.321158,-1.546906
MG,0.193421,0.553439


In [ ]:
# quando informa um intervalo, o filtro inicia no eixo das linhas
df6[1:3]

,2000,2010,2020,2030
RJ,-2.021255,-0.334077,0.002118,0.405453
SP,0.289092,1.321158,-1.546906,-0.202646


In [ ]:
# subconjunto de linhas
df6[1:3]

,2000,2010,2020,2030
RJ,-2.021255,-0.334077,0.002118,0.405453
SP,0.289092,1.321158,-1.546906,-0.202646


In [ ]:
# subconjunto de linhas e colunas (podemos misturar acesso pelo indice e pelo rotulo no mesmo comando)
df6[1:3][[2000, 2030]]

,2000,2030
RJ,-2.021255,0.405453
SP,0.289092,-0.202646


In [ ]:
# acessando dados com filtro: somente dados onde no ano de 2030 os valores são maiores que Zero
df6[df6[2030] > 0][2030]

ES    0.953324
RJ    0.405453
MG    1.318152
Name: 2030, dtype: float64

In [ ]:
# atribuindo valores Zero onde o valor for negativo
df6[df6 < 0] = 0
df6

,2000,2010,2020,2030
ES,0.000000,1.150036,0.991946,0.953324
RJ,0.000000,0.000000,0.002118,0.405453
SP,0.289092,1.321158,0.000000,0.000000
MG,0.000000,0.193421,0.553439,1.318152


### usando loc e iloc

In [ ]:
# selecionando linhas e colunas pelos rótulos: df6.loc[ [<linhas], [<colunas>] ] 
df6.loc["ES", [2000, 2030]] 

2000    0.000000
2030    0.953324
Name: ES, dtype: float64

In [ ]:
# iloc funciona da mesma forma. Mas opera com os índices implicitos que iniciam em zero
# o mesmo comando acima acessando pelos índices
df6.iloc[0, [0,3]]

2000    0.000000
2030    0.953324
Name: ES, dtype: float64

In [ ]:
# loc e iloc acessam por intervalos. Não precisam dos colchetes internos para formar os ranges
df6.loc["RJ":"MG", 2020:2030]

,2020,2030
RJ,0.002118,0.405453
SP,0.000000,0.000000
MG,0.553439,1.318152


## Cálculos e Alinhamento de Dados

In [ ]:
# dois DataFrames que compartilham algumas colunas e indices podem ser alinhados
# mesmo com shapes diferentes

data_left = pd.DataFrame(np.random.randn(9).reshape(3,3),
                         index=["ES", "RJ", "SP"],
                         columns=list("bcd"))
data_left

,b,c,d
ES,-0.469305,0.675554,-1.817027
RJ,-0.183109,1.058969,-0.397840
SP,0.337438,1.047579,1.045938


In [ ]:
# possue shape diferente
data_rigth = pd.DataFrame(np.random.randn(12).reshape(4, 3),
                          index=["SP", "RJ", "MG", "RS"],
                          columns=list("bde"))
data_rigth

,b,d,e
SP,0.863717,-0.122092,0.124713
RJ,-0.322795,0.841675,2.390961
MG,0.076200,-0.566446,0.036142
RS,-2.074978,0.247792,-0.897157


In [ ]:
# o que acontece ao somar os dos dfs com shapes diferentes
# acontece o alinhamento pelos rotulos dos indices
# se o calculo envolver algum valor NaN então retorna NaN

# só retorna algum resultado onde houver um inner join
data_left + data_rigth

,b,c,d,e
ES,NaN,NaN,NaN,NaN
MG,NaN,NaN,NaN,NaN
RJ,-0.505903,NaN,0.443834,NaN
RS,NaN,NaN,NaN,NaN
SP,1.201155,NaN,0.923847,NaN


In [ ]:
# preenche com Zero o valor NaN quando um dos elementos da soma for NaN
# quando os dois valores da soma for NaN retorna NaN
data_left.add(data_rigth, fill_value=0)

,b,c,d,e
ES,-0.469305,0.675554,-1.817027,NaN
MG,0.076200,NaN,-0.566446,0.036142
RJ,-0.505903,1.058969,0.443834,2.390961
RS,-2.074978,NaN,0.247792,-0.897157
SP,1.201155,1.047579,0.923847,0.124713


In [ ]:
# reindexa o data_left usando apenas as colunas do data_rigth
data_left.reindex(columns=data_rigth.columns, fill_value=0)

,b,d,e
ES,-0.469305,-1.817027,0
RJ,-0.183109,-0.397840,0
SP,0.337438,1.045938,0


## Operações entre DataFrames e Series

In [ ]:
df7 = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['ES', 'RJ', 'SP', 'MG'])

serie6 = df7.iloc[3]
df7

,b,d,e
ES,0.0,1.0,2.0
RJ,3.0,4.0,5.0
SP,6.0,7.0,8.0
MG,9.0,10.0,11.0


In [ ]:
serie6

b     9.0
d    10.0
e    11.0
Name: MG, dtype: float64

In [ ]:
# faz o alinhamento do índice da serie com os índice das colunas em df e faz o calculo nos correspondentes para todas as linhas
df7 - serie6

In [ ]:
# df7.sub(serie6, axis="index") ???

## Mapping  e Function

In [ ]:
df8 = pd.DataFrame(np.random.randn(12).reshape((4, 3)),
                     columns=list('bde'),
                     index=['ES', 'RJ', 'SP', 'MG'])

print(f"df8:\n {df8}\n")
np.abs(df8)

df8:
            b         d         e
ES -0.785435  0.462060  0.704228
RJ  0.523508 -0.926254  2.007843
SP  0.226963 -1.152659  0.631979
MG  0.039513  0.464392 -3.563517



,b,d,e
ES,0.785435,0.462060,0.704228
RJ,0.523508,0.926254,2.007843
SP,0.226963,1.152659,0.631979
MG,0.039513,0.464392,3.563517


In [ ]:
# calcula o valor a diferença entro o max e min de uma coluna
funcao_dif_max_min = lambda x: x.max() - x.min()
funcao_max = lambda x: x.max()
funcao_min = lambda x: x.min()

# por padrão a função é aplicada no sentido de agregação das linhas (vertical)
df8.apply(funcao_dif_max_min)

b    1.308943
d    1.617051
e    5.571360
dtype: float64

In [ ]:
df8.apply(funcao_max)

b    0.523508
d    0.464392
e    2.007843
dtype: float64

In [ ]:
df8.apply(funcao_min)

b   -0.785435
d   -1.152659
e   -3.563517
dtype: float64

In [ ]:
# aplicar o calculo no sentido das colunas (horizontal)
df8.apply(funcao_min, axis="columns")

ES   -0.785435
RJ   -0.926254
SP   -1.152659
MG   -3.563517
dtype: float64

In [ ]:
# aplicando uma função com multiplos retornos
#def f(x):
#  return pd.Series([x.max(), x.min()], index=["max", "min"])

f_lambda = lambda x: pd.Series([x.max(), x.min()], index=["max", "min"])
df8.apply(f_lambda)

,b,d,e
max,0.523508,0.464392,2.007843
min,-0.785435,-1.152659,-3.563517


In [ ]:
# applymap: aplicando uma formatação
# diferentemente de apply, applymap não faz agregação ela aplica em todas as celulas
formato = lambda x: f"R$ {x: ,.2f}"
df8.applymap(formato)

,b,d,e
ES,R$ -0.79,R$ 0.46,R$ 0.70
RJ,R$ 0.52,R$ -0.93,R$ 2.01
SP,R$ 0.23,R$ -1.15,R$ 0.63
MG,R$ 0.04,R$ 0.46,R$ -3.56


In [ ]:
# map aplica em apenas uma coluna
df8["b"].map(formato)

ES    R$ -0.79
RJ    R$  0.52
SP    R$  0.23
MG    R$  0.04
Name: b, dtype: object

## Ordenação e Rank

In [ ]:
obj_Series5 = pd.Series(range(5), index=["x", "y", "z", "a", "b"])

# ordenando uma serie pelo index
obj_Series5.sort_index()

a    3
b    4
x    0
y    1
z    2
dtype: int64

In [ ]:
# ordenando uma serie pela coluna
obj_Series5.sort_values()

x    0
y    1
z    2
a    3
b    4
dtype: int64

In [ ]:
# ordenação de índices de dataframe
df9 = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['LinhaA', 'LinhaB'],
                     columns=['d', 'a', 'b', 'c'])

# ordenando as linhas (padrao)
df9.sort_index()

# ordenando as colunas (axis=1)
df9.sort_index(axis=1)

,a,b,c,d
LinhaA,1,2,3,0
LinhaB,5,6,7,4


In [ ]:
# controlando o sentido da ordenação
df9.sort_index(axis=0, ascending=False)

,d,a,b,c
LinhaB,4,5,6,7
LinhaA,0,1,2,3


In [ ]:
# controlando o sentido da ordenação
df9.sort_index(axis=1, ascending=False)

,d,c,b,a
LinhaA,0,3,2,1
LinhaB,4,7,6,5


In [ ]:
# ordenando os valores de uma ou várias colunas de um um DataFrame
df9.sort_values(by=["c", "d"], ascending=False)

,d,a,b,c
LinhaB,4,5,6,7
LinhaA,0,1,2,3


In [ ]:
df9.sort_values(by=["LinhaB"], ascending=False, axis=1)

,c,b,a,d
LinhaA,3,2,1,0
LinhaB,7,6,5,4


## Agregação/estatísticas soma, max, min, mean, cumsum

In [ ]:
# a soma acontece no sentido das linhas (vertical) para todas as colunas
df9.sum()

d     4
a     6
b     8
c    10
dtype: int64

In [ ]:
# soma de apenas uma coluna
df9["b"].sum()

8

In [ ]:
# soma de colunas específicas
df9[["c","a"]].sum()

c    10
a     6
dtype: int64

In [ ]:
# soma no sentido horizontal, agrega as colunas para obter o todal da linha
df9.sum(axis="columns")

LinhaA     6
LinhaB    22
dtype: int64

In [ ]:
df9[:1].sum(axis="columns")

LinhaA    6
dtype: int64

In [ ]:
# outras medidas de agregação
df9.mean(axis='columns', skipna=False)

LinhaA    1.5
LinhaB    5.5
dtype: float64

In [ ]:
df9.cumsum(axis=1)

,d,a,b,c
LinhaA,0,1,3,6
LinhaB,4,9,15,22


In [ ]:
df9.describe()

,d,a,b,c
count,2.000000,2.000000,2.000000,2.000000
mean,2.000000,3.000000,4.000000,5.000000
std,2.828427,2.828427,2.828427,2.828427
min,0.000000,1.000000,2.000000,3.000000
25%,1.000000,2.000000,3.000000,4.000000
50%,2.000000,3.000000,4.000000,5.000000
75%,3.000000,4.000000,5.000000,6.000000
max,4.000000,5.000000,6.000000,7.000000


## Valores únicos value counts

In [ ]:
obj_Series6 = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj_Series6.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [ ]:
obj_Series6.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

In [ ]:
df10 = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})

df10["Qu1"].value_counts()

4    2
3    2
1    1
Name: Qu1, dtype: int64